In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from collections import defaultdict
import os, sys

mx_base = "/mnt/extproj/projekte/textmining/"
model_base = mx_base + "spacy_models/"
mirexplore_base = mx_base + "miRExplore/python/"
scai_base = mirexplore_base + "/scai_corpus/"

if not mirexplore_base in sys.path:
    sys.path.insert(0, mirexplore_base)
    

import scispacy
import spacy
from spacy import displacy

#nlp = spacy.load('/mnt/d/spacy/models/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')




In [3]:
nlp_weblg = spacy.load(model_base+"/en_core_web_lg-2.2.0/en_core_web_lg/en_core_web_lg-2.2.0/")
nlp = spacy.load(model_base+"/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/")
nlp_ent = spacy.load(model_base+"/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")

In [4]:
import textmining.MirGeneRelCheck

In [5]:

def analyse(testCase, dist=80):
    relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

    doc = nlp(testCase[0])
    
    #print("conjuncts", doc.conjuncts)
    
    lWord = doc[testCase[1]]
    rWord = doc[testCase[2]]

    for t in doc:
        print(t.idx, t.i, t, t.pos_, t.dep_, t.head, t.head.idx, t.conjuncts)

    print(lWord, rWord)

    if not "mir" in str(lWord).lower() and not "let" in str(lWord).lower():
        tmp = lWord
        lWord = rWord
        rWord = tmp
        
    print(lWord, rWord)
    
    checkResults = relCheck.checkRelation(doc, lWord, rWord, verbose=True)
    print(checkResults)
    
    displacy.render(doc, style="dep", options={"compact": True, "distance": dist})

In [6]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Disfunction of neutrophils in sepsis has been described in previous studies, however, little is known about the role of microRNA-let-7b (miR-let-7b), toll-like receptor 4 (TLR4), and nuclear factor kappa B (NF-kappaB) activity in neutrophils and how they participate in the development of sepsis.", 21, 39)

analyse(testcase)

0 0 Disfunction NOUN nsubjpass described 46 ()
12 1 of ADP case neutrophils 15 ()
15 2 neutrophils NOUN nmod Disfunction 0 ()
27 3 in ADP case sepsis 30 ()
30 4 sepsis NOUN nmod Disfunction 0 ()
37 5 has AUX aux described 46 ()
41 6 been AUX auxpass described 46 ()
46 7 described VERB ROOT described 46 ()
56 8 in ADP case studies 68 ()
59 9 previous ADJ amod studies 68 ()
68 10 studies NOUN nmod described 46 ()
75 11 , PUNCT punct known 96 ()
77 12 however ADV advmod known 96 ()
84 13 , PUNCT punct known 96 ()
86 14 little ADJ nsubjpass known 96 ()
93 15 is AUX auxpass known 96 ()
96 16 known VERB parataxis described 46 (participate,)
102 17 about ADP case role 112 ()
108 18 the DET det role 112 ()
112 19 role NOUN nmod known 96 ()
117 20 of ADP case microRNA-let-7b 120 ()
120 21 microRNA-let-7b NOUN nmod role 112 (receptor, activity)
136 22 ( PUNCT punct miR-let-7b 137 ()
137 23 miR-let-7b NOUN appos microRNA-let-7b 120 ()
147 24 ) PUNCT punct miR-let-7b 137 ()
148 25 , PUNCT punct mi

In [7]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Functional association between miR 124a and CDK6 in medulloblastoma was established using luciferase assays.", 4, 6)

analyse(testcase)

0 0 Functional ADJ amod association 11 ()
11 1 association NOUN nsubjpass established 72 ()
23 2 between ADP case miR 31 ()
31 3 miR NOUN nmod association 11 ()
35 4 124a NOUN dep miR 31 (CDK6,)
40 5 and CCONJ cc 124a 35 ()
44 6 CDK6 NOUN conj 124a 35 (124a,)
49 7 in ADP case medulloblastoma 52 ()
52 8 medulloblastoma NOUN nmod association 11 ()
68 9 was AUX auxpass established 72 ()
72 10 established VERB ROOT established 72 ()
84 11 using VERB xcomp established 72 ()
90 12 luciferase NOUN compound assays 101 ()
101 13 assays NOUN dobj using 84 ()
107 14 . PUNCT punct established 72 ()
124a CDK6
CDK6 124a
Conjuncts [CDK6, 124a]
neighbour CDK6 in
neighbour CDK6 in
Conjunction before fc 124a
Conjuncts [124a, CDK6]
neighbour 124a and
neighbour 124a and
Conjunction before fc CDK6
Conjunctions ( conj )
124a [124a, CDK6]
achildren {CDK6}
achildren False
Conjuncts [association, miR]
neighbour association between
neighbour association between
Conjunction before fc miR
fc add children for miR


In [8]:
%aimport textmining.MirGeneRelCheck

testcase = (u"Overexpression of mir-7 and mir-153 significantly reduces endogenous alpha-synuclein levels, whereas inhibition of mir-7 and mir-153 enhances translation of a luciferase construct bearing the alpha-s", 6, 10)

analyse(testcase)

0 0 Overexpression NOUN nsubj reduces 50 ()
15 1 of ADP case mir-7 18 ()
18 2 mir-7 NOUN nmod Overexpression 0 (mir-153,)
24 3 and CCONJ cc mir-7 18 ()
28 4 mir-153 NOUN conj mir-7 18 (mir-7,)
36 5 significantly ADV advmod reduces 50 ()
50 6 reduces VERB ROOT reduces 50 ()
58 7 endogenous ADJ amod levels 85 ()
69 8 alpha-synuclein ADJ amod levels 85 ()
85 9 levels NOUN dobj reduces 50 ()
91 10 , PUNCT punct reduces 50 ()
93 11 whereas SCONJ dep reduces 50 ()
101 12 inhibition NOUN nsubj enhances 133 ()
112 13 of ADP case mir-7 115 ()
115 14 mir-7 NOUN nmod inhibition 101 (mir-153,)
121 15 and CCONJ cc mir-7 115 ()
125 16 mir-153 NOUN conj mir-7 115 (mir-7,)
133 17 enhances VERB parataxis reduces 50 ()
142 18 translation NOUN dobj enhances 133 ()
154 19 of ADP case construct 170 ()
157 20 a DET det construct 170 ()
159 21 luciferase NOUN compound construct 170 ()
170 22 construct NOUN nmod translation 142 ()
180 23 bearing VERB acl construct 170 ()
188 24 the DET det alpha-s 192 ()
192 

In [9]:
analyse((u"The clinical relevance of our findings was evaluated in HIV-encephalitis (HIVE) brain samples in which decreased levels of MCP-2 and increased levels of mir-146a were observed, suggesting a role for mir-146a in the maintenance of HIV-mediated chronic inflammation of the brain.", 20, 25)
)

0 0 The DET det relevance 13 ()
4 1 clinical ADJ amod relevance 13 ()
13 2 relevance NOUN nsubjpass evaluated 43 ()
23 3 of ADP case findings 30 ()
26 4 our PRON nmod:poss findings 30 ()
30 5 findings NOUN nmod relevance 13 ()
39 6 was AUX auxpass evaluated 43 ()
43 7 evaluated VERB ROOT evaluated 43 ()
53 8 in ADP case samples 86 ()
56 9 HIV-encephalitis NOUN compound samples 86 ()
73 10 ( PUNCT punct HIVE 74 ()
74 11 HIVE NOUN appos HIV-encephalitis 56 ()
78 12 ) PUNCT punct HIVE 74 ()
80 13 brain NOUN compound samples 86 ()
86 14 samples NOUN nmod evaluated 43 ()
94 15 in ADP case which 97 ()
97 16 which PRON nmod observed 167 ()
103 17 decreased VERB amod levels 113 ()
113 18 levels NOUN nsubjpass observed 167 (levels,)
120 19 of ADP case MCP-2 123 ()
123 20 MCP-2 NOUN nmod levels 113 ()
129 21 and CCONJ cc levels 113 ()
133 22 increased VERB amod levels 143 

()
143 23 levels NOUN conj levels 113 (levels,)
150 24 of ADP case mir-146a 153 ()
153 25 mir-146a NOUN nmod levels 143 ()
162 26 were AUX auxpass observed 167 ()
167 27 observed VERB acl:relcl samples 86 ()
175 28 , PUNCT punct evaluated 43 ()
177 29 suggesting VERB xcomp evaluated 43 ()
188 30 a DET det role 190 ()
190 31 role NOUN dobj suggesting 177 ()
195 32 for ADP case mir-146a 199 ()
199 33 mir-146a NOUN nmod role 190 ()
208 34 in ADP case maintenance 215 ()
211 35 the DET det maintenance 215 ()
215 36 maintenance NOUN nmod role 190 ()
227 37 of ADP case inflammation 251 ()
230 38 HIV-mediated ADJ amod inflammation 251 ()
243 39 chronic ADJ amod inflammation 251 ()
251 40 inflammation NOUN nmod maintenance 215 ()
264 41 of ADP case brain 271 ()
267 42 the DET det brain 271 ()
271 43 brain NOUN nmod inflammation 251 ()
276 44 . PUNCT punct evaluated 43 ()
MCP-2 mir-146a
mir-146a MCP-2
Conjuncts [levels, levels]
neighbour levels of
neighbour levels of
Conjunction before fc levels

In [10]:
%aimport textmining.MirGeneRelCheck

testCase = (u"We show here that miR-29a, -29b-1, and -9 can regulate BACE1 expression in vitro.", 4, 12)
testCase = (u"It has been predicted that miR-146a may target Tata Binding Protein (TBP).",5, 12)
testCase = (u"Our results showed that the expression of miR-106b was inversely correlated with TβRII protein levels and miR-106b can directly inhibit the TβRII translation in vitro.", 7, 21)

relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

doc = nlp(testCase[0])

mirword = doc[testCase[1]]
geneword = doc[testCase[2]]

verbose=False

In [11]:
conjResult, conjs = relCheck.checkCommonConj(doc, mirword, geneword, verbose)
sdpPass, passive, negated = relCheck.checkSDP(doc, mirword, geneword, verbose)
compPass = relCheck.checkCompartments(doc, mirword, geneword, verbose)
sigPathway = relCheck.checkSurContext(doc, mirword, geneword, verbose)


In [12]:
sdpRes = relCheck._get_sdp_path(doc, mirword.i, geneword.i)
sdpRes

[(miR-106b, 'NOUN', 'nmod', False),
 (expression, 'NOUN', 'nsubjpass', False),
 (correlated, 'VERB', 'ccomp', True),
 (inhibit, 'VERB', 'conj', False),
 (translation, 'NOUN', 'dobj', False),
 (TβRII, 'NOUN', 'compound', False)]

In [13]:
%aimport textmining.MirGeneRelCheck

testCase = (u"We show here that miR-29a, -29b-1, and -9 can regulate BACE1 expression in vitro.", 4, 12)
testCase = (u"It has been predicted that miR-146a may target Tata Binding Protein (TBP).",5, 12)
testCase = (u"Our results showed that the expression of miR-106b was inversely correlated with TβRII protein levels and miR-106b can directly inhibit the TβRII translation in vitro.", 7, 21)
testCase = (u"The reduction of the affinity of miR-433 to the 3' UTR would result in increased FGF20 expression and upregulation of alpha-synuclein, which could in turn promote dopaminergic neurons degeneration.", 6, 16)
testCase = (u"These findings identify miR-29b as a novel posttranscriptional regulator of PGRN expression, raising the possibility that miR-29b or other miRNAs might be targeted therapeutically to increase hPGRN levels in some FTD patients.", 3,10)
relCheck = textmining.MirGeneRelCheck.MirGeneRelCheck()

doc = nlp(testCase[0])

mirword = doc[testCase[1]]
geneword = doc[testCase[2]]

verbose=False

from collections import Counter



In [14]:
def loadStems():
    
    print("Loading stems")
    allRels = []
    stem2class = {}

    renameClass = {
        'INT': 'NEU',
        "BOOST": "POS",
        "COMBINE": "NEU",
        "CHANGE": "NEU",
    }

    with open(mirexplore_base+"/allrels.csv") as fin:

        for line in fin:
            aline = line.strip().split(",", 1)
            aregSyns = aline[1].split(":")
            regSyns = []
            if len(aregSyns) > 1:
                regSyns = aregSyns[1].split("|")
            
            aline[1] = aline[1].split(":")[0]           

            regClass = aline[0]
            regStem = aline[1]

            if regClass in renameClass:
                regClass = renameClass[regClass]
                
            stem2class[regStem] = regClass
                
            for regSyn in regSyns:
                if not regSyn in stem2class:
                    stem2class[regSyn] = regClass


            allRels.append((regClass, regStem))

    allStems = [x for x in stem2class]
    
    return allStems, allRels, stem2class

allStems, allRels, stem2class = loadStems()


Loading stems


In [15]:
from textmining.MirGeneRelCheck import SentenceRelationChecker, SentenceRelationClassifier
from utils.tmutils import normalize_gene_names
from collections import Counter
from lxml import etree
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from itertools import chain, combinations


In [16]:
scaiBase = mirexplore_base + "/scai_corpus/"

base = "TRAIN"

if base == "TRAIN":
    scaiFile = "miRNA_train_fixed.xml"
    relsFile = "scai_train_rels.tsv"
elif base == "TEST":
    scaiFile = "miRNA_test_fixed.xml"
    relsFile = "scai_test_rels.tsv"
    
normGeneSymbols= normalize_gene_names(path=scai_base + "/hgnc_no_withdrawn.syn")

relChecker = SentenceRelationChecker(nlp)

correctIdentified = 0
incorrectIdentified = 0
totalChecks = 0
incorrectClass = Counter()


In [26]:
%aimport textmining.MirGeneRelCheck

normGeneSymbols= normalize_gene_names(path=scai_base + "/hgnc_no_withdrawn.syn")
relationsFile = mirexplore_base + "allrels.csv"



def loadRelations(relsFile):
    reldir2new = {}
    reldir2new["TARGET"] = "NEU"
    reldir2new["INVCORR"] = "NEU"
    reldir2new["REGULATE"] = "NEU"
    relClasses = {}
    with open(os.path.join(scai_base, relsFile)) as fin:
        
        print("Loading relations", relsFile)
        
        for lidx, line in enumerate(fin):

            if lidx == 0:
                continue

            line = line.strip().split("\t")

            interactionID = line[0].strip()
            relType = line[1].strip()
            relRDir = line[2].strip()
            relIndirect = line[3].strip()
            relPassive = line[4].strip()

            if relRDir in reldir2new:
                relRDir = reldir2new[relRDir]

            relClasses[interactionID] = (relType, relRDir, relIndirect, relPassive)
            
        print("Interactions", len(relClasses))
        print("Non NA Interactions", len([x for x in relClasses if not relClasses[x][0] == "NA"]))
            
    return relClasses

def all_subsets(ss):
    
    return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

def runCheck(numelems, base, relation_ids=None, original=False):
    relChecker = SentenceRelationChecker(nlp, nlp_ent=nlp_ent)
    relClassifier = SentenceRelationClassifier(relationsFile)
    
    if base == "TRAIN":
        scaiFile = "miRNA_train_fixed.xml"
        relsFile = "scai_train_rels.tsv"
    elif base == "TEST":
        scaiFile = "miRNA_test_fixed.xml"
        relsFile = "scai_test_rels.tsv"
        
    if original:
        if base == "TRAIN":
            scaiFile = "miRNA-Train-Corpus.xml"
        elif base == "TEST":
            scaiFile = "miRNA-Test-Corpus.xml"
    
    relClasses = loadRelations(relsFile)
    
    check2results = {}
    checkSubsets = [x for x in all_subsets(relClassifier.major_checks)]
    checkSubsets = [checkSubsets[-1]]
    
    print("Will test {} subsets.".format(len(checkSubsets)))
    
    for subset_checks in checkSubsets:
    
        print("Testing checks", subset_checks)
        
        #relClassifier.active_checks = subset_checks

        #allStems, allRels, stem2class = loadStems()

        with open(os.path.join(scaiBase, scaiFile), 'r') as fin:
            tree = etree.parse(fin)
            root = tree.getroot()
            scaiPairs = []
            totalChecks = 0
            correctIdentified = 0
            incorrectIdentified = 0

            incorrectClassified = 0
            correctClassified = 0
            totalClassified = 0
            correctInteractClassified = 0
            totalValidClassified = 0
            totalClassifiable = 0
            totalMirGeneDownClassified = 0

            errorByDetect = Counter()
            classifiedByDetect = Counter()
            elemCaseCounter = Counter()
            incorrectClass = Counter()

            classifyTrue = []
            classifyPred = []

            for elem in root.findall(".//document"):

                pmid = elem.attrib['origId']

                for sentElem in elem:

                    allEntities = sentElem.findall(".//entity")
                    allPairs = sentElem.findall(".//pair")

                    sentText = sentElem.attrib["text"]

                    entId2elem = {}

                    for entity in allEntities:
                        entId = entity.attrib['id']
                        entText = entity.attrib['text']
                        entType = entity.attrib['type']
                        entOffset = tuple([int(x) for x in entity.attrib['charOffset'].split("-")])

                        if entType in ["Specific_miRNAs", "Genes/Proteins"]:

                            if "Genes" in entType:
                                if entText in normGeneSymbols:
                                    entText = normGeneSymbols[entText]
                                elif entText.upper() in normGeneSymbols:
                                    gene = normGeneSymbols[entText.upper()]
                            else:
                                try:
                                    entText = miRNA(entText).getStringFromParts([miRNAPART.MATURE, miRNAPART.ID, miRNAPART.PRECURSOR])
                                except:
                                    pass

                            entTuple = (entText, entType, (entOffset[0], entOffset[1]+1))
                            entId2elem[entId] = entTuple


                            sentEntText = sentText[entTuple[2][0]:entTuple[2][1]]

                    for pair in allPairs:

                        validInteraction = pair.attrib['interaction'].lower() == "true"
                        pairE1 = pair.attrib['e1']
                        pairE2 = pair.attrib['e2']
                        relationID = pair.attrib["id"]
                        if not relation_ids is None:
                            if not relationID in relation_ids:
                                continue

                        #if pairInt == 'true':
                        if pairE1 in entId2elem and pairE2 in entId2elem:

                            totalChecks += 1

                            e1 = entId2elem[pairE1]
                            e2 = entId2elem[pairE2]

                            if not e1[1] in ["Specific_miRNAs"]:

                                tmp=e1
                                e1=e2
                                e2=tmp


                            relRes = relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=False
                                                                )


                            fullsentence = relRes['full_sentence']
                            acceptInteraction = relRes['accept_relation']

                            if not acceptInteraction == validInteraction:

                                incorrectIdentified += 1
                                
                                """
                                relResV=relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=True
                                                                )
                                
                                print(relResV)
                                """
                            else:
                                correctIdentified += 1

                            elemCaseCounter[(validInteraction, acceptInteraction)]+=1

                            totalClassified += 1

                            if validInteraction:# and acceptInteraction:
                                #print(relRes["check_results"])   
                                totalValidClassified += 1

                                foundClasses = relRes["check_results"]["classification"] #{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'regulation_keys': set()}
                                foundTuple = (foundClasses["interaction_dir"], foundClasses["regulation_dir"])                                
                                
                                testTuple = relClasses[relationID]

                                classifyTrue.append((testTuple[0], testTuple[1]))
                                classifyPred.append((foundTuple[0], foundTuple[1]))

                                if testTuple[0] == "MIR_GENE" and testTuple[1] == "DOWN":
                                    totalMirGeneDownClassified += 1

                                classifiedByDetect[foundClasses["reg_detect"]] += 1

                                if testTuple[0] != foundTuple[0] or testTuple[1] != foundTuple[1]:


                                    #print(sentText)
                                    #print(e1, e2)
                                    #print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                    #print(testTuple)

                                    print()
                                    print()
                                    print()
                                    
                                    print(relationID)
                                    print("IS: ", foundTuple)
                                    print("SB: ", testTuple)
                                    print(relRes)
                                    
                                    relResV=relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=True
                                                                )
                                
                                    print(relResV)
                                    
                                    print()
                                    print()
                                    print()

                                    errorByDetect[foundClasses["reg_detect"]] += 1
                                    
                                    

                                    incorrectClassified += 1



                                    #numelems -= 1
                                    #if numelems == 0:
                                    #    return
                                else:
                                    
                                    if foundClasses["reg_detect_major"] == "return":
                                        print(foundClasses)
                                        print(sentText)
                                        print(e1, e2)
                                        print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                        print(testTuple)

                                        print()
                                        print()
                                        print()
                                    correctClassified += 1

                                    if acceptInteraction:
                                        correctInteractClassified += 1



            def printStats(outfile):
                print("Total:     ", totalChecks, file=outfile)
                print("Correct:   ", correctIdentified, correctIdentified/totalChecks, file=outfile)
                print("Incorrect: ", incorrectIdentified, incorrectIdentified/totalChecks, file=outfile)
                print("classes", incorrectClass, file=outfile)
                
                precision = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(True, False)])
                recall = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(False, True)])

                f1 = 2* precision * recall / (precision+recall)

                specificity = elemCaseCounter[(False, False)] / (elemCaseCounter[(True, False)] + elemCaseCounter[(False, False)])

                print("precision", precision, file=outfile)
                print("recall", recall, file=outfile)
                print("specificity", specificity, file=outfile)
                print("f1", f1, file=outfile)

                print("Correct classified:     ", correctClassified, correctClassified/totalClassified, correctClassified/totalValidClassified,file=outfile)
                print("Incorrect classified:   ", incorrectClassified, incorrectClassified/totalClassified, incorrectClassified/totalValidClassified,file=outfile)
                print("Random classified: ", totalMirGeneDownClassified, totalMirGeneDownClassified / totalClassified, totalMirGeneDownClassified/totalValidClassified, file=outfile)
                print(errorByDetect)

                print("Correct interaction&classified:     ", correctInteractClassified, correctInteractClassified/totalClassified, correctInteractClassified/totalValidClassified,file=outfile)

                print("Classified by: ", classifiedByDetect)

                lClassifyTrue = ["_".join(x) for x in classifyTrue]
                lClassifyPred = ["_".join(x) for x in classifyPred]

                allLabels = sorted(set(lClassifyTrue+lClassifyPred))
                matrix = multilabel_confusion_matrix(lClassifyTrue, lClassifyPred, labels=allLabels)
                print(matrix)
                print(classification_report(lClassifyTrue,lClassifyPred))
                
                return classification_report(lClassifyTrue, lClassifyPred, output_dict=True)

            rep = printStats(sys.stdout)
            check2results[subset_checks] = rep
        #printStats(sys.stderr)
    return check2results
        


In [18]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier(relationsFile)

train_results = runCheck(-1, "TRAIN")

Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


Loading relations scai_train_rels.tsv
Interactions 397
Non NA Interactions 194
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')



miRNA-corp.d5.s2.p2
IS:  ('GENE_MIR', 'NEU')
SB:  ('MIR_GENE', 'DOWN', 'FALSE', 'FALSE')
{'accept_relation': True, 'check_results': {'conj': True, 'conj_nmod': True, 'sdp': True, 'compartment': True, 'compartment_nmod': True, 'context': True, 'entity': True, 'passive': False, 'negation': False, 'classification': {'regulation_dir': 'NEU', 'interaction_dir': 'GENE_MIR', 'passive': False, 'reg_detect': 'counts between', 'reg_detect_major': 'counts'}}, 'tagged_sent': 'Strikingly, cells with high levels of <e2>PKM2</e2> expressed lower levels of <e1>miR-326</e1>, suggestive of endogenous regulation of PKM2 by miR-326.', 'full_sentence': 'Strikingly, cells with high levels of <e2>PKM2</e2> expressed lower levels of <e1>miR-326</e1>, suggestive of endogenous regulation of PKM2 by miR-326.', 'entity1': {'entity_type': 'gene', 'enti

In [19]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier(relationsFile)

test_results = runCheck(-1, "TEST")

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'return', 'reg_detect_major': 'return'}
Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the proliferation of CRL-1690 and CRL-2610 cell lines. 
('miRNA-128', 'Specific_miRNAs', (22, 31)) ('APOBEC3C', 'Genes/Proteins', (99, 103))
('MIR_GENE', 'NEU') return passive? False
('MIR_GENE', 'NEU', 'FALSE', 'FALSE')



{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'return', 'reg_detect_major': 'return'}
Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the proliferation of CRL-1690 and CRL-2610 cell lines. 
('miRNA-128', 'Specific_miRNAs', (22, 31)) ('ANGPTL6', 'Genes/Proteins', (105, 112))
('MIR_GENE', 'NEU') retur

In [20]:
import pickle

outDirectory = mx_base + "/scai_eval/"

with open(os.path.join(outDirectory, "scai_test_f1.pickle"), "wb") as fout:
    pickle.dump(test_results, fout)

In [21]:
test_results

{('compartment',
  'between',
  'counts',
  'return'): {'GENE_MIR_NEU': {'precision': 0.6666666666666666,
   'recall': 1.0,
   'f1-score': 0.8,
   'support': 4}, 'GENE_MIR_UP': {'precision': 1.0,
   'recall': 1.0,
   'f1-score': 1.0,
   'support': 1}, 'MIR_GENE_DOWN': {'precision': 0.9074074074074074,
   'recall': 0.9074074074074074,
   'f1-score': 0.9074074074074074,
   'support': 54}, 'MIR_GENE_NEU': {'precision': 0.9615384615384616,
   'recall': 0.9259259259259259,
   'f1-score': 0.9433962264150944,
   'support': 81}, 'MIR_GENE_UP': {'precision': 0.0,
   'recall': 0.0,
   'f1-score': 0.0,
   'support': 1}, 'accuracy': 0.9148936170212766, 'macro avg': {'precision': 0.7071225071225071,
   'recall': 0.7666666666666667,
   'f1-score': 0.7301607267645004,
   'support': 141}, 'weighted avg': {'precision': 0.925895617384979,
   'recall': 0.9148936170212766,
   'f1-score': 0.9192559882242741,
   'support': 141}}}

In [22]:
import pickle

outDirectory = mx_base + "/scai_eval/"

with open(os.path.join(outDirectory, "results_test_scilg_bionlp_subsetc.pickle"), "rb") as fout:
    data = pickle.load(fout)

In [23]:
for x in data:
    print(x, data[x]["regulation"]["weighted avg"])

(('conj', 'sdp', 'compartment', 'context', 'entity'), ()) {'precision': 0.7323371771966868, 'recall': 0.45390070921985815, 'f1-score': 0.34983349025902216, 'support': 141}
(('conj', 'sdp', 'compartment', 'context', 'entity'), ('compartment',)) {'precision': 0.7659574468085106, 'recall': 0.6170212765957447, 'f1-score': 0.5879059350503919, 'support': 141}
(('conj', 'sdp', 'compartment', 'context', 'entity'), ('between',)) {'precision': 0.7134956919289609, 'recall': 0.5106382978723404, 'f1-score': 0.44811219115832557, 'support': 141}
(('conj', 'sdp', 'compartment', 'context', 'entity'), ('counts',)) {'precision': 0.8371587365562885, 'recall': 0.7943262411347518, 'f1-score': 0.8091128524634267, 'support': 141}
(('conj', 'sdp', 'compartment', 'context', 'entity'), ('return',)) {'precision': 0.35311683463979093, 'recall': 0.5815602836879432, 'f1-score': 0.4394029778798122, 'support': 141}
(('conj', 'sdp', 'compartment', 'context', 'entity'), ('compartment', 'between')) {'precision': 0.758736

In [24]:
for x in data:
    print(set(data[x]["regulation"]))

{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 'MIR_GENE_UP', 'MIR_GENE_DOWN', 'MIR_GENE_NEU', 'weighted avg', 'accuracy', 'macro avg'}
{'GENE_MIR_NEU', 'GENE_MIR_UP', 

In [27]:
%aimport textmining.MirGeneRelCheck

relClassifier = SentenceRelationClassifier(relationsFile)

test_results_original = runCheck(-1, "TEST", original=True)

Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')



miRNA-corp.d9.s1.p0
IS:  ('MIR_GENE', 'UP')
SB:  ('MIR_GENE', 'DOWN', 'TRUE', 'FALSE')
{'accept_relation': True, 'check_results': {'conj': True, 'conj_nmod': True, 'sdp': True, 'compartment': True, 'compartment_nmod': True, 'context': True, 'entity': True, 'passive': False, 'negation': False, 'classification': {'regulation_dir': 'UP', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'counts between', 'reg_detect_major': 'counts'}}, 'tagged_sent': "The reduction of the affinity of <e1>miR-433</e1> to the 3' UTR would result in increased <e2>FGF20</e2> expression and upregulation of alpha-synuclein, which could in turn promote dopaminergic neurons degeneration.", 'full_sentence': "The reduction of the affinity of <e1>miR-433</e1> to the 3' UTR would result in increased <e2>FGF20</e2> expression and upregulation o

/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/si